In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import joblib
import os

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


In [2]:
# Cargar el dataset desde la carpeta 'data'
df = pd.read_csv('data/tramites_municipales.csv')

print("Dataset cargado. Dimensiones:", df.shape)
print("\nPrimeras 5 filas:")
df.head()

Dataset cargado. Dimensiones: (2500, 12)

Primeras 5 filas:


,id_tramite,nombre_solicitante,dni_solicitante,fecha_solicitud,tipo_tramite,area_responsable,canal_entrada,monto_asociado,documentos_completos,solicitante_declara_urgencia,dias_desde_solicitud,prioridad
0,1000,Lisandro Alcolea Abascal,10.48272984/j1b4n57,2025-06-08,Pago de Impuestos,Desarrollo Urbano,Telefónico,7934.15,True,False,23,Media
1,1001,Chucho Manrique Pazos,10.16377566/f4c1e36,2025-06-03,Licencia de Funcionamiento,Registro Civil,En Línea,1463.53,True,False,28,Media
2,1002,Julio César Serrano Quintanilla,10.63487675/f9s3r88,2025-06-01,Licencia de Construcción,Registro Civil,En Línea,199935.21,True,False,30,Media
3,1003,Amor Gascón Perez,10.99668400/d3y5e38,2025-05-19,Solicitud de Partida de Nacimiento,Registro Civil,Presencial,0.00,True,False,43,Media
4,1004,Lázaro Sacristán Galindo,10.1503773/c4u0j81,2025-06-27,Licencia de Funcionamiento,Desarrollo Urbano,Presencial,1599.90,True,False,4,Baja


In [3]:
# La variable que queremos predecir es 'prioridad'
y = df['prioridad']

# Las características (features) son todas las columnas relevantes que ayudan a predecir la prioridad.
# Excluimos las que no aportan información predictiva general.
columnas_a_eliminar = [
    'id_tramite',           # Es un simple identificador
    'nombre_solicitante',   # El nombre no debe influir en la prioridad
    'dni_solicitante',      # El DNI tampoco debe influir
    'fecha_solicitud',      # Ya tenemos 'dias_desde_solicitud' que es más útil
    'prioridad'             # Esta es nuestra variable objetivo, no una característica
]

X = df.drop(columns=columnas_a_eliminar)

print("--- Características (X) que se usarán para el entrenamiento ---")
print(X.columns.to_list())
print("\n--- Primeras filas de X ---")
X.head()

--- Características (X) que se usarán para el entrenamiento ---
['tipo_tramite', 'area_responsable', 'canal_entrada', 'monto_asociado', 'documentos_completos', 'solicitante_declara_urgencia', 'dias_desde_solicitud']

--- Primeras filas de X ---


,tipo_tramite,area_responsable,canal_entrada,monto_asociado,documentos_completos,solicitante_declara_urgencia,dias_desde_solicitud
0,Pago de Impuestos,Desarrollo Urbano,Telefónico,7934.15,True,False,23
1,Licencia de Funcionamiento,Registro Civil,En Línea,1463.53,True,False,28
2,Licencia de Construcción,Registro Civil,En Línea,199935.21,True,False,30
3,Solicitud de Partida de Nacimiento,Registro Civil,Presencial,0.00,True,False,43
4,Licencia de Funcionamiento,Desarrollo Urbano,Presencial,1599.90,True,False,4


In [4]:
# Identificar automáticamente las columnas categóricas (texto) y numéricas
categorical_features = X.select_dtypes(include=['object', 'category']).columns
numeric_features = X.select_dtypes(include=['int64', 'float64', 'bool']).columns

print("Columnas Categóricas detectadas:", list(categorical_features))
print("Columnas Numéricas/Booleanas detectadas:", list(numeric_features))

# Crear un pipeline para las transformaciones numéricas (escalado)
# y otro para las categóricas (one-hot encoding)
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore', drop='first')

# Unir estos transformadores en un único preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Crear el pipeline completo que une el preprocesador y el modelo
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(random_state=42)) # Ya no necesitaríamos max_iter
])

print("\nPipeline creado exitosamente.")

Columnas Categóricas detectadas: ['tipo_tramite', 'area_responsable', 'canal_entrada']
Columnas Numéricas/Booleanas detectadas: ['monto_asociado', 'documentos_completos', 'solicitante_declara_urgencia', 'dias_desde_solicitud']

Pipeline creado exitosamente.


In [5]:
print("Entrenando el modelo con todos los datos...")

# Entrenar el pipeline completo
model_pipeline.fit(X, y)

print("\n¡Modelo entrenado exitosamente!")

Entrenando el modelo con todos los datos...

¡Modelo entrenado exitosamente!


In [6]:
# Predecir sobre los mismos datos de entrenamiento para ver qué tan bien se ajusta
y_pred = model_pipeline.predict(X)

print("--- Reporte de Clasificación (sobre datos de entrenamiento) ---")
print(classification_report(y, y_pred))

--- Reporte de Clasificación (sobre datos de entrenamiento) ---
              precision    recall  f1-score   support

        Alta       0.90      0.78      0.84       301
        Baja       0.99      0.98      0.99       386
       Media       0.94      0.98      0.96      1013
     Urgente       0.98      0.99      0.99       800

    accuracy                           0.96      2500
   macro avg       0.96      0.93      0.94      2500
weighted avg       0.96      0.96      0.96      2500



In [7]:
# Crear el directorio 'modelo/' si no existe
output_dir = 'modelo'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Guardar el pipeline completo en un archivo
model_path = os.path.join(output_dir, 'modelo_prioridad.pkl')
joblib.dump(model_pipeline, model_path)

print(f"\n✅ Modelo guardado exitosamente en: '{model_path}'")


✅ Modelo guardado exitosamente en: 'modelo\modelo_prioridad.pkl'
